In [29]:
import sys
import os
from glob import glob
from pathlib import Path
from collections import defaultdict
import shutil

import ecif
import Bio
from Bio.PDB import PDBParser
import mol2parser
import progressbar
import pandas as pd
import numpy as np
from rdkit import Chem
from scipy import spatial
from xgboost import XGBRegressor
from scipy.stats import pearsonr, spearmanr, kendalltau
from sklearn.metrics import mean_squared_error
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from molfeat.trans.pretrained.hf_transformers import PretrainedHFTransformer

pd.options.mode.chained_assignment = None
%matplotlib inline

*Python and Libraries Versions*:

**Python Version**: 3.8.16

**Biopython version**: 1.77

**Progressbar version**: 4.2.0

**Pandas version**: 1.5.3

**Numpy version**: 1.23.5

**RDKit version**: 2022.09.5

**Scipy version**: 1.10.1

**XGBoost version**: 1.7.6

**Sklearn version**: 1.2.1

**Matplotlib version**: 3.7.1

**MolFeat version**: 0.8.5

**Jupyter Lab**: 3.6.1



**Protein-Ligand Complexes Directory**

In [ ]:
# determine folders which your PL complexes are located.

pdbbind_refined_set_2020_path = r"\refined-set_2020"
casf_2016_path = r"\CASF-2016"
feature_vector_directory = r".\files\feature_vector"

# 1- Feature Generation Functions

## 1-1- Protein-Ligand Complexes

### 1-1-1- RF-Score and GB-Score

In [ ]:
#Parse a pdb file and output a dictionary contains AA classes with their elemental atom types.
#mol2 file is parsed by mol2parser (https://github.com/miladrayka/mol2parser).

def pdb_parser(pdbfile, amino_acid_classes):

    number_of_groups = len(list(amino_acid_classes))

    amino_acid_groups = zip([f"group{i}" for i in range(1, number_of_groups + 1)],
                            amino_acid_classes)

    parser = PDBParser(PERMISSIVE=True, QUIET=True)
    protein = parser.get_structure("", pdbfile)

    extracted_protein_atoms = {f"group{i}": defaultdict(
        list) for i in range(1, number_of_groups + 1)}

    for name, group in amino_acid_groups:
        for residue in protein.get_residues():
            if residue.get_resname() in group:
                for atom in residue.get_atoms():
                    extracted_protein_atoms[name][atom.element].append(
                        list(atom.coord))

    return extracted_protein_atoms

In [ ]:
# Give a weight to a contact between two atom types based on their distance using 1/d**2 term 
#(used in GB-Score) and sums all same pair.

def weighting_sum(distances, cutoff, exp):

    selected_distances = distances[distances < cutoff]

    feature = sum(list(map(lambda x: 1.0 / (x ** exp), selected_distances)))

    return feature

# Count same pair atom types contact (used in RF-Score). 

def counting_sum(distances, cutoff):
    
    selected_distances = distances[distances < cutoff]
    
    feature = len(selected_distances)
    
    return feature

In [ ]:
#Feature generation function for both RF-Score (feature_type="counting_sum") 
#and GB-Score (exp=2, feature_type="weighting_sum"). 

def features_generator(mol2file, pdbfile, amino_acid_classes, cutoff, feature_type, exp=None):

    ligand = mol2parser.Mol2Parser(mol2file)
    ligand.parse()

    extracted_protein_atoms = pdb_parser(pdbfile, amino_acid_classes)

    groups = extracted_protein_atoms.keys()
    feature_vector = defaultdict(list)

    for group in groups:
        for protein_element in protein_elements:
            for ligand_element in ligand_elements:

                try:
                    ligand_coords = np.array(ligand.get_molecule(
                        ligand_element, "coords"), dtype="float").reshape(-1, 3)

                except TypeError:
                    continue

                if extracted_protein_atoms[group][protein_element]:
                    protein_coords = np.array(
                        extracted_protein_atoms[group][protein_element], dtype="float")

                    distances = spatial.distance.cdist(
                        ligand_coords, protein_coords).ravel()

                    if feature_type == "weighting_sum":
                        feature = weighting_sum(distances, cutoff, exp)
                        
                    if feature_type == "counting_sum":
                        feature = counting_sum(distances, cutoff)

                    if len(groups) > 1:
                        name = group + "_" + protein_element + "_" + ligand_element
                        
                    else:
                        name = protein_element + "_" + ligand_element
                        
                    feature_vector[name].append(feature)

    return feature_vector

In [ ]:
#Generate feature vector for a directory of protein-ligand complexes.

def complexes_feature_generator(pathfiles, amino_acid_classes, 
                                cutoff, feature_type, exp=None):
    
    entries = Path(pathfiles)
    numbers = len(os.listdir(pathfiles))

    bar = progressbar.ProgressBar(maxval=numbers).start()

    feature_vectors = {}

    for num, entry in enumerate(entries.iterdir()):
    
        num += 1

        bar.update(num)

        ligand_file = glob(str(entry) + "\*_ligand.mol2")[0]
        protein_file = glob(str(entry) + "\*_protein.pdb")[0]

        try: 
            fv = features_generator(ligand_file, protein_file,
                                amino_acid_classes, cutoff, 
                                feature_type, exp=exp)
            feature_vectors[entry.name] = fv
        
        except Exception:
            continue
            
    return feature_vectors

In [ ]:
#Save generated feature vectors to csv for both RF-Score and GB-Score.

def save_to_csv(feature_vectors, filename):
    
    df = pd.DataFrame.from_dict(feature_vectors, orient="index")
    df = df.applymap(lambda x: x[0], na_action="ignore")
    df = df.fillna(0.0)
    
    df.to_csv(filename, index=True, index_label="pdbid")

### 1-1-2- ECIF

In [ ]:
#Generate feature vector for a directory of protein-ligand complexes based on ECIF.
#ECIF code is located at https://github.com/DIFACQUIM/ECIF.

def complexes_feature_generator_ecif(pathfiles):
    
    entries = Path(pathfiles)
    numbers = len(os.listdir(pathfiles))

    bar = progressbar.ProgressBar(maxval=numbers).start()

    ecif_feature_vectors = {}
    ligand_descriptor_feature_vectors = {}

    for num, entry in enumerate(entries.iterdir()):
    
        num += 1

        bar.update(num)

        ligand_file = glob(str(entry) + "\*_ligand.sdf")[0]
        protein_file = glob(str(entry) + "\*_protein.pdb")[0]

        try: 
            ECIF = ecif.GetECIF(protein_file, ligand_file, distance_cutoff=6.0)
            ligand_descriptors = ecif.GetRDKitDescriptors(ligand_file)
            ecif_feature_vectors[entry.name] = ECIF
            ligand_descriptor_feature_vectors[entry.name] = ligand_descriptors
        
        except Exception:
            continue
        
    return ligand_descriptor_feature_vectors, ecif_feature_vectors

In [ ]:
#Save generated feature vectors to csv for ECIF.

def save_to_csv_ecif(ligand_descriptor_feature_vectors, ecif_feature_vectors, filename):
    
    df2 = pd.DataFrame.from_dict(ligand_descriptor_feature_vectors, orient="index", columns=ecif.LigandDescriptors)
    df1 = pd.DataFrame.from_dict(ecif_feature_vectors, orient="index", columns=ecif.PossibleECIF)
    df3 = df1.join(df2)
    
    df3.to_csv(filename, index=True, index_label="pdbid")

## 1-2- Ligand 

In [ ]:
# Generate feature vectors for ligand. 

def ligands_feature_generator(pathfiles, kind):

    entries = Path(pathfiles)
    numbers = len(os.listdir(pathfiles))

    bar = progressbar.ProgressBar(maxval=numbers).start()

    ligand_descriptor_feature_vectors = {}

    for num, entry in enumerate(entries.iterdir()):

        num += 1

        bar.update(num)

        ligand_file = glob(str(entry) + "\*_ligand.sdf")[0]

        if kind == "rdkit":

            try:
                
                ligand_descriptors = ecif.GetRDKitDescriptors(ligand_file)
                ligand_descriptor_feature_vectors[entry.name] = ligand_descriptors

            except Exception:
                continue

        if kind == "ChemBERTa-77M-MLM":
            
            try:
                mol = Chem.MolFromMolFile(ligand_file, sanitize=False)
                mol.UpdatePropertyCache(strict=False)
                
                ligand_descriptors = transformer_chembert(mol)
                ligand_descriptor_feature_vectors[entry.name] = ligand_descriptors[0]
                
            except Exception:
                continue

    return ligand_descriptor_feature_vectors

In [ ]:
#Save generated feature vectors to csv for ligand's descriptors.

def save_to_csv_ligand(ligand_descriptor_feature_vectors, kind, filename):
    
    if kind == "rdkit":
        
        df = pd.DataFrame.from_dict(ligand_descriptor_feature_vectors, orient="index", columns=ecif.LigandDescriptors)
      
    if kind == "ChemBERTa-77M-MLM":
        
        df = pd.DataFrame.from_dict(ligand_descriptor_feature_vectors, orient="index")  
        
    df.to_csv(filename, index=True, index_label="pdbid")

# 2- PDBbind 2020

## 2-1- Generate Feature Vectors

**Necessarry Information**

In [ ]:
# GB-Score amino acid classification.

amino_acid_classes_1 = [
    ["ARG", "LYS", "ASP", "GLU"],
    ["GLN", "ASN", "HIS", "SER", "THR", "CYS"],
    ["TRP", "TYR", "MET"],
    ["ILE", "LEU", "PHE", "VAL", "PRO", "GLY", "ALA"],
]

# RF-Score amino acids classification (All AA in one class) 

amino_acid_classes_2 = [["ARG", "LYS", "ASP", "GLU", "GLN", "ASN", "HIS", 
                        "SER", "THR", "CYS", "TRP", "TYR", "MET","ILE", 
                        "LEU", "PHE", "VAL", "PRO", "GLY", "ALA"]]

# Elemental atom types for ligand and protein. 

ligand_elements = ["H", "C", "N", "O", "S", "P", "F", "Cl", "Br", "I"]

protein_elements = ["H", "C", "N", "O", "S"]

### 2-1-1- RF-Score Features

In [ ]:
rf_score_features = complexes_feature_generator(pdbbind_refined_set_2020_path,
                                                amino_acid_classes_2,
                                                12,
                                                "counting_sum",
                                                exp=None)

save_to_csv(rf_score_features, feature_vector_directory + "\\rfscore_refined_set_2020.csv")

### 2-1-2- GB-Score Features

In [ ]:
gb_score_features = complexes_feature_generator(pdbbind_refined_set_2020_path, 
                                                amino_acid_classes_1,
                                                12,
                                                "weighting_sum",
                                                exp=2)

save_to_csv(gb_score_features, feature_vector_directory + "\\gbscore_refined_set_2020.csv")

### 2-1-3- ECIF Features

In [ ]:
ligand_descriptor_feature_vectors, ecif_feature_vectors = complexes_feature_generator_ecif(pdbbind_refined_set_2020_path)

save_to_csv_ecif(ligand_descriptor_feature_vectors, 
                 ecif_feature_vectors, 
                 feature_vector_directory + "\\ecif_refined_set_2020.csv")

### 2-1-4- RDKit Descriptors

In [ ]:
ligand_rdkit_feature_vectors = ligands_feature_generator(pdbbind_refined_set_2020_path, kind="rdkit")

save_to_csv_ligand(ligand_rdkit_feature_vectors,
                   kind = "rdkit",
                   filename = feature_vector_directory + "\\ligand_rdkit_refined_set_2020.csv")

### 2-1-5- ChemBERTa-77M-MLM Features

In [ ]:
transformer_chembert = PretrainedHFTransformer(kind='ChemBERTa-77M-MLM', dtype=float)

In [ ]:
ligand_bert_feature_vectors = ligands_feature_generator(pdbbind_refined_set_2020_path, kind="ChemBERTa-77M-MLM")

save_to_csv_ligand(ligand_bert_feature_vectors,
                   kind = "ChemBERTa-77M-MLM",
                   filename = feature_vector_directory + "\\ligand_chembert_refined_set_2020.csv")

### 2-1-6- Water Features

In [ ]:
# Water features are generated based on the following paper:
# J. Chem. Inf. Model. 2022, 62, 4369−4379 (https://doi.org/10.1021/acs.jcim.2c00916)
# HydroMap 1.0 is used o predict potential water molecules in the protein binding pocket (http://www.sioc-ccbg.ac.cn/software/hydramap/):
# 1- Place all _ligand.mol2, _ligand.sdf, and _protein.pdb files in a single folder (e.g. D:\refined-set_2020-together)
# 2- Place gen_wat4.sh, wsitem, and getwat in the mentioned folder 
# 3- Run the following commands in WSL cmd:
# cd "/mnt/d/refined-set_2020-together"
# ./gen_wat4.sh .
# 4- At the end, for each complex a pdbid_wat4.pdb is generated.
# After that, water features is generated by using HydraMapSF calFeature.py at https://github.com/xiaoyangqu/HydraMapSF:
# 0- Create a new env: mamba create --name hydramap python=3.7 (Install the following library: Pandas, SciPy, RDkit (Mamba), PyMol(2.5.5)(Mamba))
# 1- Activate hydramap: mamba activate hydramap
# 2- Edit list_exp_2020, list_pdbid, and "test_dir=" at the end of the calFeature.py script and change function key (feat="hydra") and put
# list_pdbid and list_exp_2020 in "D:\refined-set_2020-together".
# 3- Run the following command: python calFeature.py 
# 4- Finally a file with the name hydra_11.5_1.0 is generated.
# Caution: During water feature generation binding affinity values add to .csv file. Delete Them.

In [ ]:
# Make a "refined-set_2020-together" directory.

destination_path = r"\refined-set_2020-together"
entries = Path(pdbbind_refined_set_2020_path)

In [ ]:
file = open("files\list_pdbid.txt", "a")

for num, entry in enumerate(entries.iterdir()):
    
    file.write(entry.name + "\n")
    
    ligand_mol2_path = entry.joinpath(entry.name + "_ligand.mol2")
    ligand_sdf_path = entry.joinpath(entry.name + "_ligand.sdf")
    protein_pdb_path = entry.joinpath(entry.name + "_protein.pdb")
    
    shutil.copy2(ligand_mol2_path, destination_path)
    shutil.copy2(ligand_sdf_path, destination_path)
    shutil.copy2(protein_pdb_path, destination_path)

file.close()

In [ ]:
df = pd.read_csv("files\refined_set_2020_binding_data.csv").set_index("pdbid")

In [ ]:
with open("files\list_pdbid.txt", "r") as file:
    
    pdbids = file.readlines()
    
pdbids = [item.strip() for item in pdbids]

In [ ]:
df.loc[pdbids, :].to_csv("files\list_exp_2020.csv")

In [ ]:
wat_feat_df = pd.read_csv("hydra_11.5_1.csv", axis=0, ignore_index=True).drop(columns=["binding_affinity"])
wat_feat_df.to_csv(feature_vector_directory + "\water_feature_refined_set_2020.csv", index=False)

## 2-2- Training, Validating, and Testing

In [ ]:
feature_vector_directory = r".\files\feature_vector"
water_features_csv = feature_vector_directory + "\\water_feature_refined_set_2020_full.csv"
rfscore_features_csv = feature_vector_directory + "\\rfscore_refined_set_2020_full.csv"
gbscore_features_csv = feature_vector_directory + "\\gbscore_refined_set_2020_full.csv"
ecif_features_csv = feature_vector_directory + "\\ecif_refined_set_2020_full.csv"
rdkit_features_csv = feature_vector_directory + "\\ligand_rdkit_refined_set_2020_full.csv"
chembert_features_csv = feature_vector_directory + "\\ligand_chembert_refined_set_2020_full.csv"
binding_affinity_csv = ".\files\refined_set_2020_binding_data_full.csv"
core_set_2016_csv = ".\files\casf_core_set_2016.csv"
drive_path = ".\files\trained_model_and_results\"

### 2-2-0- PDBbind 2020 Analysis

In [ ]:
df = pd.read_csv(binding_affinity_csv)

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(df["binding_affinity"], bins=12, rwidth=0.8, color="dodgerblue", edgecolor="navy")
plt.xlabel("Binding Constant (-$logk_a$)", fontsize=14)
plt.ylabel("Occurrence", fontsize=14)
plt.title("Distribution of the Protein−Ligand Binding Constants (5335 Complexes)", fontsize=16)
plt.grid(True, ls="--", lw=0.7, c="slategrey")
plt.savefig("Distribution of the Protein−Ligand Binding Constants.png", dpi=300)
plt.show()

In [ ]:
rdkit_df = pd.read_csv(rdkit_features_csv)

In [ ]:
axes = rdkit_df.loc[:, ["MolWt", "MolLogP", "HeavyAtomCount", "NumRotatableBonds", "NumHAcceptors", "NumHDonors"]].hist(bins=12, 
                                                                                                                        rwidth=0.8, 
                                                                                                                        figsize=(10, 8), 
                                                                                                                        color="dodgerblue", 
                                                                                                                        edgecolor="navy",
                                                                                                                        )
for axe in axes.flatten():
    axe.set_ylabel("Occurrence")
plt.tight_layout()
plt.savefig("Distribution of Some Key Properties of Ligand.png", dpi=300)
plt.show()

### 2-2-1- Feature Processing Functions

In [ ]:
def preprocessing(data, var_threshold=0.01, corr_threshold=0.95):

    """
    Preprocess features input pd.DataFrame and drop static, quasi-static and
    correlated features. Return normalized and processed data in
    pd.DataFrame, mean and std of data.

     Parameters:
        data (pd.DataFrame): Features file in pd.DataFrame.
        var_threshold (float): Variance threshold. Features below this
        threshold are discarded.
        corr_threshold (float): Correlated features are discarded.

     Returns:
        data, mean, std (tuple): Return processed features and mean and std of all features.
    """

    data = data.loc[:, data.var(axis=0) > var_threshold]

    corr_matrix = data.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [
        column for column in upper.columns if any(upper[column] > corr_threshold)
    ]
    data = data.drop(to_drop, axis=1)

    mean = data.mean()
    std = data.std()
    data = (data - mean) / std

    return data, mean, std

In [ ]:
def data_spliter(data, core_set_id, val_set_size=0):

    """
    Using "core set id" to splitted data to train and test set.
    return dictionary contains train and test set (and val set) features and targets.

     Parameters:
        data (pd.DataFrame): Features file in pd.DataFrame.
        core_set_id (csv): A csv file contain all pdbid of the core set (PDBbind 2016)
        val_set_size (float): If not zero, indicate the percentage of data in validation set.
        Validation set is created randomly with the stratified split method.

     Returns:
        sets (dict): Return dictionary contains train and test set (and val set) features and targets.
    """

    test_set = data.loc[core_set_id, :]
    train_set = data.drop(core_set_id, axis=0)

    if val_set_size:

        train_set["ba_cat"] = np.ceil(train_set["binding_affinity"] / 1.5)
        train_set["ba_cat"].where(train_set["ba_cat"] < 8, 8, inplace=True)

        split = StratifiedShuffleSplit(
            n_splits=10, test_size=val_set_size, random_state=42
        )

        for train_index, val_index in split.split(train_set, train_set["ba_cat"]):

            strat_train_set = train_set.iloc[list(train_index), :]
            strat_val_set = train_set.iloc[list(val_index), :]

        strat_train_set.drop(["ba_cat"], axis=1, inplace=True)
        strat_val_set.drop(["ba_cat"], axis=1, inplace=True)

        x_train = strat_train_set.iloc[:, :-1]
        y_train = strat_train_set.iloc[:, -1]
        x_val = strat_val_set.iloc[:, :-1]
        y_val = strat_val_set.iloc[:, -1]
        x_test = test_set.iloc[:, :-1]
        y_test = test_set.iloc[:, -1]

        sets = {
            "x_train": x_train,
            "y_train": y_train,
            "x_val": x_val,
            "y_val": y_val,
            "x_test": x_test,
            "y_test": y_test,
        }
    else:

        x_train = train_set.iloc[:, :-1]
        y_train = train_set.iloc[:, -1]
        x_test = test_set.iloc[:, :-1]
        y_test = test_set.iloc[:, -1]

        sets = {
            "x_train": x_train,
            "y_train": y_train,
            "x_test": x_test,
            "y_test": y_test,
        }

    return sets


In [ ]:
def data_creator(binding_affinity_csv,
                 core_set_2016_csv,
                 structure_based_feat=None, 
                 ligand_based_feat=None,
                 ligand_based_feat_name=None,
                 water_based_feat=None,
                 var_threshold=0.01, 
                 corr_threshold=0.95,
                 val_set_size=0.1
                ):
    
    
    core_set_id = list(pd.read_csv(core_set_2016_csv)["pdbid"])
    y = pd.read_csv(binding_affinity_csv).set_index("pdbid")
    
    X = pd.read_csv(structure_based_feat).set_index("pdbid")
    
    X_sb_processed, X_sb_mean, X_sb_std = preprocessing(X, 
                                                        var_threshold, 
                                                        corr_threshold)
        
    if water_based_feat and ligand_based_feat:
        
        
        X = pd.read_csv(ligand_based_feat).set_index("pdbid")
        
        if ligand_based_feat_name == "RDKit":
            
            X_lb_processed, X_lb_mean, X_lb_std = preprocessing(X, 
                                                                var_threshold, 
                                                                corr_threshold)
        else:
            
            X_lb_processed = X
        
        X = pd.read_csv(water_based_feat).set_index("pdbid")
        X_wb_processed, X_wb_mean, X_wb_std = preprocessing(X, 
                                                            var_threshold, 
                                                            corr_threshold)
        
        data = pd.concat([X_sb_processed, X_lb_processed, X_wb_processed, y], axis=1)
        
    elif ligand_based_feat:
        
        X = pd.read_csv(ligand_based_feat).set_index("pdbid")
        
        if ligand_based_feat_name == "RDKit":
            
            X_lb_processed, X_lb_mean, X_lb_std = preprocessing(X, 
                                                                var_threshold, 
                                                                corr_threshold)
        else:
            
            X_lb_processed = X
        
        data = pd.concat([X_sb_processed, X_lb_processed, y], axis=1)
        
    else:
        
        data = pd.concat([X_sb_processed, y], axis=1)
        
    
    sets = data_spliter(data, core_set_id, val_set_size=val_set_size)
    
    return sets
    

### 2-2-2- Learning 

In [ ]:
datasets = data_creator(binding_affinity_csv,
                   core_set_2016_csv,
                   structure_based_feat=ecif_features_csv,
                   ligand_based_feat=None,
                   ligand_based_feat_name=None,
                   water_based_feat=None,
                   var_threshold=0.01,
                   corr_threshold=0.95,
                   val_set_size=0.1)

In [ ]:
"GBScore", "RFScore", "ECIF", "RDKit", "ChemBert", "HydraMap"

In [ ]:
structure_based_feature_name = "RFScore"
ligand_based_feature_name = "ChemBert"
water_based_feature_name = "HydraMap"
name = structure_based_feature_name + "_" + ligand_based_feature_name + "_" + water_based_feature_name
name

In [ ]:
for i in range(1, 11):

    print(f"Model: {name}_{i}")

    size = dataset['x_train'].shape[0]

    dataset['x_train_subsample'] = dataset['x_train'].sample(int(size * 0.9))
    dataset['y_train_subsample'] = dataset['y_train'].loc[list(dataset['x_train_subsample'].index)]
    
    xgb_reg = XGBRegressor(
            n_estimators=20000,
            max_depth=8,
            learning_rate=0.005,
            tree_method="gpu_hist",
            predictor="gpu_predictor",
            n_jobs=2
            )

    xgb_reg.fit(dataset["x_train_subsample"], dataset["y_train_subsample"])

    y_pred_test = xgb_reg.predict(dataset["x_test"])
    pd.DataFrame(y_pred_test, dataset["x_test"].index, 
             columns=["binding_affinity_pred"]).to_csv(drive_path + f"ba_test_pred_{name}_{i}.csv")
    rp = pearsonr(dataset["y_test"], y_pred_test)[0]
    rmse = np.sqrt(mean_squared_error(dataset["y_test"], y_pred_test))
    print(f"Test - Rp: {rp:.3f} RMSE: {rmse:.3f}")
    
    y_pred_val = xgb_reg.predict(dataset["x_val"])
    pd.DataFrame(y_pred_val, dataset["x_val"].index, 
             columns=["binding_affinity_pred"]).to_csv(drive_path + f"ba_val_pred_{name}_{i}.csv")
    rp = pearsonr(dataset["y_val"], y_pred_val)[0]
    rmse = np.sqrt(mean_squared_error(dataset["y_val"], y_pred_val))
    print(f"Validation - Rp: {rp:.3f} RMSE: {rmse:.3f}\n")

    xgb_reg.save_model(drive_path + f"xgb_{name}_{i}.json")

    print("---------------------------------------------")

### 2-2-3- Scoring Power Results

In [ ]:
for i in ["GBScore", "RFScore", "ECIF"]:
    for j in ["RDKit", "ChemBert", "None"]:
        for k in ["HydraMap", "None"]:
            name = i + "_" + j + "_" + k

            y_pred_test_all = []
            y_pred_val_all = []
            rp_test_all = []
            rmse_test_all = []
            rp_val_all = []
            rmse_val_all = []

            for item in range(1, 11):

                y_pred_test_df = pd.read_csv(drive_path + f"ba_test_pred_{name}_{item}.csv")
                y_pred_val_df = pd.read_csv(drive_path + f"ba_val_pred_{name}_{item}.csv")

                rp = pearsonr(dataset["y_test"], y_pred_test_df["binding_affinity_pred"])[0]
                rmse = np.sqrt(mean_squared_error(dataset["y_test"], y_pred_test_df["binding_affinity_pred"]))
                rp_test_all.append(rp)
                rmse_test_all.append(rmse)
                y_pred_test_all.append(y_pred_test_df["binding_affinity_pred"])

                rp = pearsonr(dataset["y_val"], y_pred_val_df["binding_affinity_pred"])[0]
                rmse = np.sqrt(mean_squared_error(dataset["y_val"], y_pred_val_df["binding_affinity_pred"]))
                y_pred_val_all.append(y_pred_val_df["binding_affinity_pred"])
                rp_val_all.append(rp)
                rmse_val_all.append(rmse)

            rp_test_all.append(np.mean(rp_test_all))
            rmse_test_all.append(np.mean(rmse_test_all))
            rp_val_all.append(np.mean(rp_val_all))
            rmse_val_all.append(np.mean(rmse_val_all))

            rp = pearsonr(dataset["y_test"], np.mean(y_pred_test_all, axis=0))[0]
            rmse = np.sqrt(mean_squared_error(dataset["y_test"], np.mean(y_pred_test_all, axis=0)))

            rp_test_all.append(rp)
            rmse_test_all.append(rmse)

            rp = pearsonr(dataset["y_val"],np.mean(y_pred_val_all, axis=0))[0]
            rmse = np.sqrt(mean_squared_error(dataset["y_val"], np.mean(y_pred_val_all, axis=0)))

            rp_val_all.append(rp)
            rmse_val_all.append(rmse)

            df = pd.DataFrame([rp_test_all, rmse_test_all, rp_val_all, rmse_val_all], 
             index=["RP_Test", "RMSE_Test", "RP_Val", "RMSE_Val"]).T.round(3)

            df.to_csv(drive_path + name + "_metrics.csv")

In [ ]:
df_list = []

for i in ["GBScore", "RFScore", "ECIF"]:
    for j in ["RDKit", "ChemBert", "None"]:
        for k in ["HydraMap", "None"]:

            name = i + "_" + j + "_" + k
            name_mean = name + "_mean"
            name_ens = name + "_ens"

            df = pd.read_csv(drive_path + name + "_metrics.csv", index_col=0).iloc[-2:, :]
            df.rename(index={ 10: name_mean, 11: name_ens}, inplace=True)
            df_list.append(df)

metric_df = pd.concat(df_list)

In [ ]:
metric_df.sort_values(by=["RP_Test"], ascending=False, inplace=True)

In [ ]:
dataset = data_creator(binding_affinity_csv,
                   core_set_2016_csv,
                   structure_based_feat=rfscore_features_csv,
                   ligand_based_feat=rdkit_features_csv,
                   ligand_based_feat_name="RDKit",
                   water_based_feat=water_features_csv,
                   var_threshold=0.01,
                   corr_threshold=0.95,
                   val_set_size=0.1)

In [ ]:
y_pred_test_all = []
y_pred_val_all = []

for name in ["ECIF_RDKit_HydraMap", "GBScore_ChemBert_HydraMap", "RFScore_RDKit_HydraMap"]:

    for item in range(1, 11):
    
        y_pred_test_df = pd.read_csv(drive_path + f"ba_test_pred_{name}_{item}.csv")
        y_pred_val_df = pd.read_csv(drive_path + f"ba_val_pred_{name}_{item}.csv")

        y_pred_test_all.append(y_pred_test_df["binding_affinity_pred"])
        y_pred_val_all.append(y_pred_val_df["binding_affinity_pred"])
        

rp = pearsonr(dataset["y_test"], np.mean(y_pred_test_all, axis=0))[0]
rmse = np.sqrt(mean_squared_error(dataset["y_test"], np.mean(y_pred_test_all, axis=0)))
print(f"Test - Rp: {rp:.3f} RMSE: {rmse:.3f}")

rp = pearsonr(dataset["y_val"], np.mean(y_pred_val_all, axis=0))[0]
rmse = np.sqrt(mean_squared_error(dataset["y_val"], np.mean(y_pred_val_all, axis=0)))
print(f"Val - Rp: {rp:.3f} RMSE: {rmse:.3f}")

In [ ]:
# for correlation of residual errors and fragment structure of ligand part.

df = pd.DataFrame(np.mean(y_pred_test_all, axis=0), list(dataset["y_test"].index), columns=["Score(pK)"]).reset_index()
df["Ligand"] = df["index"] + "_ligand"
df["Receptor"] = df["index"] + "_protein"
df.drop("index", axis=1, inplace=True)
df = df.iloc[:, [2, 1, 0]]
df.to_csv(drive_path + "ens-score_scores.csv", index=True, index_label="Index")

In [ ]:
pd.DataFrame(zip(np.mean(y_pred_test_all, axis=0).round(2), dataset["y_test"].tolist()), 
             columns=["Target", "Predicted"]).to_csv(drive_path + "ens_score_results_test_set.csv", index=False)

In [ ]:
pd.DataFrame(zip(np.mean(y_pred_val_all, axis=0).round(2), dataset["y_val"].tolist()), 
             columns=["Target", "Predicted"]).to_csv(drive_path + "ens_score_results_val_set.csv", index=False)

In [ ]:
sf1 = y_pred_test_all[:10]
sf2 = y_pred_test_all[10:20]
sf3 = y_pred_test_all[20:]

In [ ]:
perf_list = defaultdict(list)

for item in range(1, 11):
    
    rp_sf1 = pearsonr(dataset["y_test"], np.mean(sf1[:item], axis=0))[0]
    rmse_sf1 = np.sqrt(mean_squared_error(dataset["y_test"], np.mean(sf1[:item], axis=0)))
    
    perf_list["rp_sf1"].append(rp_sf1)
    perf_list["rmse_sf1"].append(rmse_sf1)
    
    rp_sf2 = pearsonr(dataset["y_test"], np.mean(sf2[:item], axis=0))[0]
    rmse_sf2 = np.sqrt(mean_squared_error(dataset["y_test"], np.mean(sf2[:item], axis=0)))
    
    perf_list["rp_sf2"].append(rp_sf2)
    perf_list["rmse_sf2"].append(rmse_sf2)
    
    rp_sf3 = pearsonr(dataset["y_test"], np.mean(sf3[:item], axis=0))[0]
    rmse_sf3 = np.sqrt(mean_squared_error(dataset["y_test"], np.mean(sf3[:item], axis=0)))
    
    perf_list["rp_sf3"].append(rp_sf3)
    perf_list["rmse_sf3"].append(rmse_sf3)    

In [ ]:
plt.plot(range(1, 11), perf_list["rp_sf1"], lw=2, color="crimson", label="ECIF: RDKit: HydraMap: ens")
plt.plot(range(1, 11), perf_list["rp_sf2"], lw=2, color="deepskyblue", label="DWIC: ChemBert: HydraMap: ens")
plt.plot(range(1, 11), perf_list["rp_sf3"], lw=2, color="forestgreen", label="OIC: RDKit: HydraMap: ens")
plt.xlabel("Number of Models", fontsize=12)
plt.ylabel("$R_{P}$", fontsize=12)
plt.ylim(0.8, 0.85)
plt.xticks(range(1, 11))
plt.grid(True, ls="--", lw=0.7, c="slategrey")
plt.tight_layout()
plt.legend()
plt.savefig("Rp vs Number of Models.png", dpi=300)
plt.show()

In [ ]:
plt.plot(range(1, 11), perf_list["rmse_sf1"], lw=2, color="crimson", label="ECIF: RDKit: HydraMap: ens")
plt.plot(range(1, 11), perf_list["rmse_sf2"], lw=2, color="deepskyblue", label="DWIC: ChemBert: HydraMap: ens")
plt.plot(range(1, 11), perf_list["rmse_sf3"], lw=2, color="forestgreen", label="OIC: RDKit: HydraMap: ens")
plt.xlabel("Number of Models", fontsize=12)
plt.ylabel("$RMSE$", fontsize=12)
plt.ylim(1.20, 1.40)
plt.xticks(range(1, 11))
plt.grid(True, ls="--", lw=0.7, c="slategrey")
plt.tight_layout()
plt.legend()
plt.savefig("RMSE vs Number of Models.png", dpi=300)
plt.show()

In [ ]:
plt.text(9.5, 2, "$R_P$ = 0.842 \n$RMSE$ = 1.285", fontsize=9)
plt.scatter(dataset["y_test"], np.mean(y_pred_test_all, axis=0), s=30, c="mediumseagreen", linewidth=0.7, edgecolor="black")
plt.xlabel("Predicted Values", fontsize=11)
plt.ylabel("Experimental Values", fontsize=12)
plt.yticks(range(1, 13))
plt.xticks(range(1, 13))
plt.grid(True, ls="--", lw=0.7, c="slategrey")
plt.title("Experimental vs Predicted Binding Affinity (Core Set 2016)")
plt.gca().set_aspect('equal')
plt.tight_layout()
plt.savefig("Experimental vs Predicted Binding Affinity (Core Set 2016).png", dpi=300)
plt.show()

In [ ]:
plt.text(9.2, 2, "$R_P$ = 0.846 \n$RMSE$ = 1.048", fontsize=9)
plt.scatter(dataset["y_val"], np.mean(y_pred_val_all, axis=0), s=30, c="steelblue", linewidth=0.7, edgecolor="black")
plt.xlabel("Predicted Values", fontsize=11)
plt.ylabel("Experimental Values", fontsize=12)
plt.yticks(range(1, 13))
plt.xticks(range(1, 13))
plt.grid(True, ls="--", lw=0.7, c="slategrey")
plt.title("Experimental vs Predicted Binding Affinity (Validation Set)")
plt.gca().set_aspect('equal')
plt.tight_layout()
plt.savefig("Experimental vs Predicted Binding Affinity (Validation Set).png", dpi=300)
plt.show()

In [ ]:
#"GBScore" = "distance-weighted interatomic contacts" = "DWIC"
#"RFScore" = "occurrence of interatomic contacts" = "OIC"
#"extensive connectivity of interatomic fingerprints" = "ECIF"
# e.g. DWIC:RDKit:HydraMap:mean

In [ ]:
df = pd.read_csv(drive_path + "metrics_refined_set_2020.csv")
df = df.apply(lambda x: x.replace({"RFScore": "OIC", "GBScore": "DWIC", "_": ": "}, regex=True))
df.set_index("Unnamed: 0").to_csv(drive_path + "metrics_refined_set_2020_modif_names.csv")

In [ ]:
df = pd.read_csv(drive_path + "metrics_refined_set_2020_modif_names.csv")

In [ ]:
models_dwic_based = ["DWIC: None: None: mean", 
                     "DWIC: None: None: ens", 
                     "DWIC: None: HydraMap: mean", 
                     "DWIC: None: HydraMap: ens",
                     "DWIC: RDKit: None: mean", 
                     "DWIC: RDKit: None: ens",
                     "DWIC: ChemBert: None: mean",
                     "DWIC: ChemBert: None: ens",
                     "DWIC: RDKit: HydraMap: mean",
                     "DWIC: RDKit: HydraMap: ens",
                     "DWIC: ChemBert: HydraMap: mean",
                     "DWIC: ChemBert: HydraMap: ens"]

dwic_metrics = {"rp_test":[0.795, 0.812, 0.797, 0.813, 0.786, 0.802, 0.793, 0.809, 0.805, 0.819, 0.809, 0.825],
                "rmse_test": [1.366, 1.337, 1.360, 1.333, 1.383, 1.355, 1.390, 1.365, 1.347, 1.322, 1.358, 1.334],
                "rp_val": [0.792, 0.806, 0.791, 0.805, 0.802, 0.815, 0.799, 0.812, 0.803, 0.815, 0.804, 0.816],
                "rmse_val": [1.167, 1.134, 1.168, 1.136, 1.142, 1.110, 1.148, 1.117, 1.140, 1.112, 1.138, 1.111]}

models_oic_based = ["OIC: None: None: mean", 
                    "OIC: None: None: ens", 
                    "OIC: None: HydraMap: mean", 
                    "OIC: None: HydraMap: ens",
                    "OIC: RDKit: None: mean", 
                    "OIC: RDKit: None: ens",
                    "OIC: ChemBert: None: mean",
                    "OIC: ChemBert: None: ens",
                    "OIC: RDKit: HydraMap: mean",
                    "OIC: RDKit: HydraMap: ens",
                    "OIC: ChemBert: HydraMap: mean",
                    "OIC: ChemBert: HydraMap: ens"]

oic_metrics = {"rp_test":[0.744, 0.765, 0.742, 0.764, 0.770, 0.786, 0.760, 0.776, 0.810, 0.824, 0.802, 0.816],
               "rmse_test": [1.473, 1.438, 1.476, 1.440, 1.419, 1.392, 1.450, 1.426, 1.336, 1.311, 1.371, 1.350],
               "rp_val": [0.743, 0.760, 0.747, 0.765, 0.784, 0.799, 0.766, 0.781, 0.796, 0.808, 0.793, 0.804],
               "rmse_val": [1.281, 1.240, 1.270, 1.229, 1.185, 1.151, 1.227, 1.194, 1.157, 1.130, 1.166, 1.139]}

models_ecif_based = ["ECIF: None: None: mean", 
                     "ECIF: None: None: ens", 
                     "ECIF: None: HydraMap: mean", 
                     "ECIF: None: HydraMap: ens",
                     "ECIF: RDKit: None: mean", 
                     "ECIF: RDKit: None: ens",
                     "ECIF: ChemBert: None: mean",
                     "ECIF: ChemBert: None: ens",
                     "ECIF: RDKit: HydraMap: mean",
                     "ECIF: RDKit: HydraMap: ens",
                     "ECIF: ChemBert: HydraMap: mean",
                     "ECIF: ChemBert: HydraMap: ens"]

ecif_metrics = {"rp_test":[0.788, 0.808, 0.786, 0.804, 0.797, 0.816, 0.771, 0.791, 0.821, 0.837, 0.814, 0.830],
                "rmse_test": [1.388, 1.356, 1.393, 1.363, 1.360, 1.328, 1.439, 1.409, 1.300, 1.274, 1.341, 1.316],
                "rp_val": [0.766, 0.781, 0.765, 0.779, 0.771, 0.784, 0.754, 0.768, 0.768, 0.780, 0.755, 0.767],
                "rmse_val": [1.227, 1.194, 1.231, 1.197, 1.218, 1.186, 1.255, 1.224, 1.223, 1.195, 1.251, 1.226]}

In [ ]:
colors = ["indianred"] * 12
colors[::2] = ["lightblue"] * 6
edgecolors = ["crimson"] * 12
edgecolors[::2] = ["darkcyan"] * 6

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True, figsize=(10, 8), layout="constrained")
fig.suptitle('Models $R_P$ Values (Core Set 2016)', fontsize=14)
fig.text(0.5, 0, "Models", fontsize=14)
num = 0
axes[num].set_ylabel("$R_P$", fontsize=14)
for item in zip([models_oic_based, models_dwic_based, models_ecif_based], [oic_metrics, dwic_metrics, ecif_metrics]):
        
    axes[num].bar(item[0], item[1]["rp_test"], width=0.8, color=colors, edgecolor=edgecolors)
    axes[num].set_ylim(0, 1)
    axes[num].tick_params(labelrotation=90)
    axes[num].grid(True, ls="--", lw=0.7, c="slategrey")
        
    num += 1

plt.savefig("Models R_P Values (Core Set 2016).png", dpi=300)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True, figsize=(10, 8), layout="constrained")
fig.suptitle('Models $R_P$ Values (Validation Set)', fontsize=14)
fig.text(0.5, 0, "Models", fontsize=14)
num = 0
axes[num].set_ylabel("$R_P$", fontsize=14)
for item in zip([models_oic_based, models_dwic_based, models_ecif_based], [oic_metrics, dwic_metrics, ecif_metrics]):
        
    axes[num].bar(item[0], item[1]["rp_val"], width=0.8, color=colors, edgecolor=edgecolors)
    axes[num].set_ylim(0, 1)
    axes[num].tick_params(labelrotation=90)
    axes[num].grid(True, ls="--", lw=0.7, c="slategrey")
        
    num += 1

plt.savefig("Models R_P Values (Validation Set).png", dpi=300)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True, figsize=(10, 8), layout="constrained")
fig.suptitle('Models $RMSE$ Values (Core Set 2016)', fontsize=14)
fig.text(0.5, 0, "Models", fontsize=14)
num = 0
axes[num].set_ylabel("$RMSE$", fontsize=14)
for item in zip([models_oic_based, models_dwic_based, models_ecif_based], [oic_metrics, dwic_metrics, ecif_metrics]):
        
    axes[num].bar(item[0], item[1]["rmse_test"], width=0.8, color=colors, edgecolor=edgecolors)
    axes[num].set_ylim(0, 1.6)
    axes[num].tick_params(labelrotation=90)
    axes[num].grid(True, ls="--", lw=0.7, c="slategrey")
        
    num += 1

plt.savefig("Models RMSE Values (Core Set 2016).png", dpi=300)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True, figsize=(10, 8), layout="constrained")
fig.suptitle('Models $RMSE$ Values (Validation Set)', fontsize=14)
fig.text(0.5, 0, "Models", fontsize=14)
num = 0
axes[num].set_ylabel("$RMSE$", fontsize=14)
for item in zip([models_oic_based, models_dwic_based, models_ecif_based], [oic_metrics, dwic_metrics, ecif_metrics]):
        
    axes[num].bar(item[0], item[1]["rmse_val"], width=0.8, color=colors, edgecolor=edgecolors)
    axes[num].set_ylim(0, 1.6)
    axes[num].tick_params(labelrotation=90)
    axes[num].grid(True, ls="--", lw=0.7, c="slategrey")
        
    num += 1

plt.savefig("Models RMSE Values (Validation Set).png", dpi=300)
plt.show()

# 3- Uncertainty Quantification

In [ ]:
feature_vector_directory = r".\files\feature_vector"
water_features_csv = feature_vector_directory + "\\water_feature_refined_set_2020_full.csv"
rfscore_features_csv = feature_vector_directory + "\\rfscore_refined_set_2020_full.csv"
gbscore_features_csv = feature_vector_directory + "\\gbscore_refined_set_2020_full.csv"
ecif_features_csv = feature_vector_directory + "\\ecif_refined_set_2020_full.csv"
rdkit_features_csv = feature_vector_directory + "\\ligand_rdkit_refined_set_2020_full.csv"
chembert_features_csv = feature_vector_directory + "\\ligand_chembert_refined_set_2020_full.csv"
binding_affinity_csv = ".\files\refined_set_2020_binding_data_full.csv"
core_set_2016_csv = ".\files\casf_core_set_2016.csv"

In [ ]:
dataset = data_creator(binding_affinity_csv,
                   core_set_2016_csv,
                   structure_based_feat=rfscore_features_csv,
                   ligand_based_feat=rdkit_features_csv,
                   ligand_based_feat_name="RDKit",
                   water_based_feat=water_features_csv,
                   var_threshold=0.01,
                   corr_threshold=0.95,
                   val_set_size=0.1)

In [ ]:
drive_path = "refined_set_2020_results\\"
y_pred_test_all = []
y_pred_val_all = []

for name in ["ECIF_RDKit_HydraMap", "GBScore_ChemBert_HydraMap", "RFScore_RDKit_HydraMap"]:

    for item in range(1, 11):
    
        y_pred_test_df = pd.read_csv(drive_path + f"ba_test_pred_{name}_{item}.csv")
        y_pred_val_df = pd.read_csv(drive_path + f"ba_val_pred_{name}_{item}.csv")

        y_pred_test_all.append(y_pred_test_df["binding_affinity_pred"])
        y_pred_val_all.append(y_pred_val_df["binding_affinity_pred"])
        

y_pred_val_ensemble_mean = np.mean(y_pred_val_all, axis=0)
y_pred_val_ensemble_std = np.std(y_pred_val_all, axis=0)

y_pred_test_ensemble_mean = np.mean(y_pred_test_all, axis=0)
y_pred_test_ensemble_std = np.std(y_pred_test_all, axis=0)

In [ ]:
nonconformity_values = abs((y_pred_val_ensemble_mean - np.array(dataset['y_val']))/np.exp(y_pred_val_ensemble_std))
nonconformity_values.sort()
per_80 = np.percentile(nonconformity_values, 80)

print(f"80th percentile: {per_80:.3f}")

In [ ]:
confidence_region = np.exp(y_pred_test_ensemble_std) * per_80

In [ ]:
result_cp_df = pd.DataFrame({"Confidence Region": confidence_region,
              "Mean Prediction": y_pred_test_ensemble_mean,
              "Max Prediction": y_pred_test_ensemble_mean + confidence_region,
              "Min Prediction": y_pred_test_ensemble_mean - confidence_region,
              "Experimental": dataset['y_test']})

In [ ]:
result_cp_df.to_csv(drive_path + "CI_CASF2016.csv")
result_cp_df.describe().to_csv(drive_path + "CI_statistics_CASF2016.csv")

In [ ]:
result_cp_df80 = result_cp_df[(result_cp_df['Experimental']<result_cp_df['Max Prediction'])
                       & (result_cp_df['Experimental']>result_cp_df['Min Prediction'])]

num = result_cp_df80.shape[0] 
print(f"Number of test data in confidence interval: {num} or {num/285:.2f}%")

In [ ]:
result_cp_df80_2 = result_cp_df[(result_cp_df['Confidence Region']<1.2) & 
                (result_cp_df['Experimental']<result_cp_df['Max Prediction']) &
                (result_cp_df['Experimental']>result_cp_df['Min Prediction'])]

num = result_cp_df80_2.shape[0] 
print(f"Number of test data in confidence interval below 2.0: {num} or {num/285:.2f}")

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(result_cp_df['Confidence Region'], color="mediumseagreen", edgecolor="darkslategrey", linewidth=1.7)
plt.xlabel("Size of Confidence Region", fontsize=14)
plt.ylabel("Number of Data", fontsize=14)
plt.grid(True, ls="--", lw=0.9, c="darkgrey")
plt.savefig("hist_CI_CASF2016.png", dpi=300)
plt.show()

In [ ]:
plt.text(9.5, 2, "$R_P$ = 0.842 \n$RMSE$ = 1.285", fontsize=9)
plt.scatter(result_cp_df["Experimental"], y_pred_test_ensemble_mean, s=30, c=result_cp_df["Confidence Region"].tolist(),  cmap=matplotlib.colormaps['plasma'])
plt.xlabel("Predicted Values", fontsize=11)
plt.ylabel("Experimental Values", fontsize=12)
plt.colorbar(label="Size of Confidence Region")
plt.yticks(range(1, 13))
plt.xticks(range(1, 13))
plt.grid(True, ls="--", lw=0.7, c="slategrey")
plt.gca().set_aspect('equal')
plt.tight_layout()
plt.savefig("Experimental vs Predicted Binding Affinity with Confidence Values (Core Set 2016).png", dpi=300)
plt.show()

In [ ]:
plt.scatter(result_cp_df["Experimental"], result_cp_df["Confidence Region"], s=30, c=result_cp_df["Confidence Region"], cmap=matplotlib.colormaps['plasma'])
plt.xlabel("Experimental Values", fontsize=11)
plt.ylabel("Confidence Values", fontsize=12)
plt.colorbar(label="Size of Confidence Region", orientation='horizontal')
plt.yticks(range(1, 5))
#plt.xticks(range(1, 13))
plt.grid(True, ls="--", lw=0.7, c="slategrey")
plt.gca().set_aspect('equal')
plt.tight_layout()
plt.savefig("Confidence Values vs Experimental Values (Core Set 2016).png", dpi=300)
plt.show()

In [ ]:
spearmanr(result_cp_df["Experimental"], result_cp_df["Confidence Region"])[0]

In [ ]:
df = pd.read_csv(drive_path + "CI_CASF2016.csv", index_col=0)

In [ ]:
df[df["Confidence Region"] < 1.2].shape[0] / 285

In [ ]:
df[df["Confidence Region"] < 1.2].shape[0]